In [0]:
import os
!rm -rf *`
!rm -rf .kaggle/
!git clone https://github.com/bkahn-github/data-science-bowl.git
!pip install -r data-science-bowl/requirements.txt
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip3 install Pillow
!mkdir ~/.kaggle
f = open(".kaggle/kaggle.json", "w")
f.write('{"username":"bkkaggle","key":"bf4e62680116284087ace0484990f87a"}')
f.close()
!kaggle competitions download -c data-science-bowl-2018
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_train.zip -d ~/.kaggle/competitions/data-science-bowl-2018/stage1_train
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_test.zip -d ~/.kaggle/competitions/data-science-bowl-2018/stage1_test
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage2_test_final.zip -d ~/.kaggle/competitions/data-science-bowl-2018/stage2_test
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_train_labels.csv.zip -d ~/.kaggle/competitions/data-science-bowl-2018/
!unzip ~/.kaggle/competitions/data-science-bowl-2018/stage1_sample_submission.csv.zip -d ~/.kaggle/competitions/data-science-bowl-2018/

In [0]:
# !git clone https://github.com/lopuhin/kaggle-dsbowl-2018-dataset-fixes ./kaggle-dsbowl-2018-dataset-fixes
# !ln -s ./kaggle-dsbowl-2018-dataset-fixes/stage1_train stage1_train
# !mv ~/.kaggle/competitions/data-science-bowl-2018/test/ stage1_test
# !mv ~/.kaggle/competitions/data-science-bowl-2018/stage1_train_labels.csv .
# !mv ~/.kaggle/competitions/data-science-bowl-2018/stage1_sample_submission.csv .

In [0]:
!git clone https://github.com/waleedka/coco.git

In [0]:
!apt-get install make

In [0]:
os.chdir('coco/PythonAPI/')
!pip install Cython
!make install

In [0]:
os.chdir('../../')

In [0]:
!git clone https://github.com/bkahn-github/Mask_RCNN.git
os.chdir('./Mask_RCNN')

In [0]:
os.chdir('./samples/nucleus/')

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1bl9uBz4DP3xeTX0BnE35jEvVfFW__Ybh'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model.h5')

In [0]:
!pip install imgaug

In [0]:
# !python3 nucleus.py train --dataset='/content/.kaggle/competitions/data-science-bowl-2018/' --subset=train --weights='./model.h5'

In [0]:
!python3 nucleus.py detect --dataset='/content/.kaggle/competitions/data-science-bowl-2018/' --subset=stage2_test --weights='./model.h5'

In [0]:
!mv ../../logs/ saved.h5

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
uploaded = drive.CreateFile({'title': 'matterport-45-epochs.h5'})
uploaded.SetContentFile('../../logs/nucleus20180411T1538/mask_rcnn_nucleus_0010.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
from google.colab import files
files.download('../../results/nucleus/submit_20180413T181459/submit.csv')

In [0]:
len(glob('../../results/nucleus/submit_20180412T031048/*'))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage import io
from skimage.segmentation import mark_boundaries
from glob import glob
from matplotlib.colors import ListedColormap
import re

%matplotlib inline

In [0]:
def show_image(image_id, title=None, path='../../../.kaggle/competitions/data-science-bowl-2018/stage2_test'):
    """
    Plot colored nuclea on top of the image
    """
    image = io.imread(os.path.join(path, image_id, 'images', image_id + '.png'))[:, :, :3]
    masks = glob(os.path.join(path, image_id, 'masks', '*.png'))
    
    rand_cmap = ListedColormap(np.random.rand(len(masks), 3))
    masked_canvas = np.zeros(image.shape[:3])
    
    for i, mask_path in enumerate(glob(os.path.join(path, image_id, 'masks', '*.png'))):
        mask = io.imread(mask_path, as_grey=True)
        masked_canvas[mask > 0] = i
        
    plt.figure(figsize=(20, 20))
    transparent_canvas = np.where(masked_canvas > 0, masked_canvas, np.nan)
    
    if title:
        plt.title(title)        
    
    plt.imshow(image)
    
    plt.figure(figsize=(20, 20))
    plt.imshow(image)
    plt.imshow(transparent_canvas, cmap=rand_cmap)
    
def show_images(image_ids, path='stage1_train'):
    for image_id in image_ids:
        show_image(image_id, path)

def show_images_with_title(df, path='stage1_train'):
    for idx, row in df.iterrows():
        show_image(row['image_id'], row['image_id'] + ', ' + str(row['mask_count']) + ' masks', path)
        
def build_labels_df_from_path(path):
    ids = []
    id_extractor = re.compile(path + '\/(?P<image_id>.*)\/masks\/(?P<mask_id>.*)\.png')
    for path in glob(path + '/**/masks/*.png'):
        matches = id_extractor.match(path)
        image_id = matches.group('image_id')
        mask_id = matches.group('mask_id')
        mask = io.imread(path, mode='L')
        ids.append([image_id, mask_id, mask.shape[0], mask.shape[1]])
        
    return pd.DataFrame(ids, columns=['image_id', 'mask_id', 'height', 'width'])

def build_test_df_from_path(path):
    ids = []
    id_extractor = re.compile(path + '\/(?P<image_id>.*)\/images\/.*\.png')
    for path in glob(path + '/**/images/*.png'):
        matches = id_extractor.match(path)
        image_id = matches.group('image_id')
        image = io.imread(path, mode='L')
        ids.append([image_id, image.shape[0], image.shape[1]])
        
    return pd.DataFrame(ids, columns=['image_id', 'height', 'width'])

In [0]:
train = io.imread_collection('../../../.kaggle/competitions/data-science-bowl-2018/stage1_train/**/images/*.png')
test = io.imread_collection('../../../.kaggle/competitions/data-science-bowl-2018/stage2_test/**/images/*.png')
train_masks = io.imread_collection('../../../.kaggle/competitions/data-science-bowl-2018/stage1_train/**/masks/*.png')

In [0]:
len(train), len(test), len(train_masks)

In [0]:
train_labels_df = build_labels_df_from_path('../../../.kaggle/competitions/data-science-bowl-2018/stage1_train')

In [0]:
train_labels_df.shape

In [0]:
train_labels_df.head()

In [0]:
train_labels_df[['height', 'width']].describe()

In [0]:
test_df = build_test_df_from_path('../../../.kaggle/competitions/data-science-bowl-2018/stage2_test')

In [0]:
test_df.shape

In [0]:
test_df.head()

In [0]:
test_df[['height', 'width']].describe()

In [0]:
mask_counts = train_labels_df.groupby('image_id')[['mask_id']].count()
mask_counts = mask_counts.reset_index()
mask_counts.columns = ['image_id', 'mask_count']

In [0]:
mask_counts.sort_values('mask_count', ascending=False)

In [0]:
mask_counts['mask_count'].hist(bins=70)

In [0]:
test_df['image_id'][0]

In [0]:
show_images(test_df['image_id'][100:200], path='../../../.kaggle/competitions/data-science-bowl-2018/stage2_test')